In [1]:
import numpy as np
import numpy.linalg as la
import numpy.random as ra
import matplotlib.pyplot as plt
from PIL import Image

N = 100
m = 2*np.sqrt(2/np.pi)*np.log(N)

In [2]:
## Cholesky Decomposition of Covariance Structure of the Grid

L = np.zeros((N**2, N**2))

for i in range(N):
    for j in range(N):
        for k in range(N):
            for l in range(N):
                if (i,j) == (k,l):
                    L[N*i+j, N*k+l] = 4
                if ((k == i-1 or k == i+1) and l == j) or ((l == j-1 or l == j+1) and i == k):
                    L[N*i+j, N*k+l] = -1

G = la.inv(L)
A = la.cholesky(G)

np.save('cholesky'+str(N), A)

In [2]:
A = np.load('cholesky'+str(N)+'.npy')

In [3]:
## Ornstein-Uhlenbeck Bridge

def OU(start, T, dt):
    X = np.zeros(int(T/dt)+1)
    X[0] = start
    for i in range(1, int(T/dt)+1):
        X[i] = X[i-1] * (1 - 0.5*dt) + ra.normal(scale=np.sqrt(dt))
    return X

def OUBridge(T, dt):
    finished = False
    
    while not finished:
        start = ra.normal(scale=1)
        X1 = OU(start, T, dt)
        X2 = np.flip(OU(start, T, dt))
        
        indexofintersection = -1
        
        for i in range(1,len(X1)):
            if (not finished) and np.sign(X1[i-1] - X2[i-1]) != np.sign(X1[i] - X2[i]):
                indexofintersection = i
                finished = True
                
    return np.concatenate((X1[:indexofintersection], X2[indexofintersection:]))[:-1]

In [ ]:
## The actual image generation code
T = 30
tps = 100

# double-check this!
noise = np.array([OUBridge(T, 1/tps) for _ in range(N**2)])
gffs = np.transpose(np.dot(A, noise)).reshape((T * tps, N, N))

## get HSV color data
rawhue = (gffs + 0.333*m)/(0.666*m) * 270/360
h = np.maximum(np.minimum(rawhue, 270/360), 0)
s = np.minimum(270/360 + np.maximum(rawhue-h, np.maximum(h-rawhue, 0)), 1)
v = np.broadcast_to(1, gffs.shape)

## convert it to RGB
i = np.floor(h * 6)
f = h * 6 - i
w = v * (1 - s)
q = v * (1 - s * f)
t = v * (1 - s * (1 - f))

i0 = np.float64(i==0)
i1 = np.float64(i==1)
i2 = np.float64(i==2)
i3 = np.float64(i==3)
i4 = np.float64(i==4)
i5 = np.float64(i==5)

r = v*i0 + q*i1 + w*i2 + w*i3 + t*i4 + v*i5
g = t*i0 + v*i1 + v*i2 + q*i3 + w*i4 + w*i5
b = w*i0 + w*i1 + t*i2 + v*i3 + v*i4 + q*i5

# generate frames
colordata = np.uint8(255*np.stack((r,g,b), axis=-1))
frames = [Image.fromarray(c) for c in colordata]

frames[0].save('test11.gif', save_all=True, append_images=frames[1:], duration=T*tps/50, loop=0)